## Коррекция опечаток

([Задание для семинара](#scrollTo=17IUm4LgTykL), [Домашнее задание](#scrollTo=vHh31ssMCZ-w&line=12&uniqifier=1))

При наборе текста автор может допустить опечатки, орфографические  и грамматические ошибки в словах. У специальных терминов также может  существовать несколько допустимых  форм написания. Рассмотрим задачу исправления  ошибочного написания слова, которое  можно обнаружить сверкой
со словарем.

Рассмотрим, какие методы используются для коррекции опечаток, какие мультиязычные и русскоязычные библиотеки можно использовать.

ПРИМЕР (из [Dialogue evaluation 2016](http://www.dialog-21.ru/en/evaluation/2016/spelling_correction/) ):

неправильно
>вот в инете откапал такую интеерсную статейку предлагаю вашему внимани

правильно
>вот в инете **откопал** такую **интересную** статейку предлагаю вашему **вниманию**


Пусть слова из обрабатываемого текста необходимо сверить с некоторым словарем.  Существует два основных способа сверки:
 1. Сгенерировать множество модификаций слова и найти эти модификации в словаре; 
 2. Найти в словаре наиболее похожие токены на основе некоторое метрики сходства (расстояние редактирования, расстояние Хэмминга, близость векторов и т.д.).

Из найденного набора вариантов для замены неправильного слова необходимо найти  наиболее вероятный для данного контекста (например, на основе частотности словарного  слова в языке или синтаксиса).

### 1. Базовый способ

**Задание (Семинар, 2 балла):** Дополните код функции, возвращающей самый частотный из наиболее похожих на данное слово слов словаря. 

Простейший подход к исправлению слов: находим в словаре наиболее похожие токены, из них выбираем самый вероятный (например, самый частотный) для замены некорректно напечатанного.

В качестве примера возьмем какой-нибудь готовый частотный словарь для русского языка, например, [такой](https://github.com/Baksalyar/mc.hertzbeat.ru-Frequency-Dictionaries).

In [1]:
! wget -q https://github.com/Baksalyar/mc.hertzbeat.ru-Frequency-Dictionaries/raw/master/mc.hertzbeat.ru_frequency_dict.txt

In [1]:
freq_dict = []

with open("mc.hertzbeat.ru_frequency_dict.txt", "r") as f:
    for string in f:
        freq_dict.append((string.split()[0], int(string.split()[1].strip())))

In [2]:
freq_dict[:10]

[('в', 1546309),
 ('и', 1124915),
 ('на', 779113),
 ('не', 471559),
 ('с', 467340),
 ('что', 441934),
 ('по', 328627),
 ('для', 225411),
 ('а', 206812),
 ('из', 204946)]

Базовая метрика близости токенов: **Расстояние Дамерау-Левенштейна** или редакционное расстояние с учетом транспозиции.

Эта функция есть в ``nltk``: ``nltk.metrics.edit_distance``. Код приведен для ясности.

In [3]:
def damerau_levenshtein_distance(s1, s2, transposition=True):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if transposition:
                if i and j and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                    d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + cost) # transposition
 
    return d[lenstr1-1,lenstr2-1]

In [4]:
wrong_text, true_text = "вот в инете откапал такую интеерсную статейку предлагаю вашему внимани", "вот в инете откопал такую интересную статейку предлагаю вашему вниманию"
wrong, true = wrong_text.split(), true_text.split()

In [5]:
for i in range(len(true)):
    if wrong[i] != true[i]:
        dld = damerau_levenshtein_distance(wrong[i], true[i], transposition=False)
        print(f"Levenshtein distance between {wrong[i]} and {true[i]} = {dld}")

Levenshtein distance between откапал and откопал = 1
Levenshtein distance between интеерсную and интересную = 2
Levenshtein distance between внимани and вниманию = 1


In [6]:
for i in range(len(true)):
    if wrong[i] != true[i]:
        dld = damerau_levenshtein_distance(wrong[i], true[i], transposition=True)
        print(f"Damerau–Levenshtein distance between {wrong[i]} and {true[i]} = {dld}")

Damerau–Levenshtein distance between откапал and откопал = 1
Damerau–Levenshtein distance between интеерсную and интересную = 1
Damerau–Levenshtein distance between внимани and вниманию = 1


In [7]:
def in_dict(token):
    for s in freq_dict:
        if s[0] == token:
            return token
    
def dummy_spellcorrection(token, max_dist=2):
    if in_dict(token):
        return token
    #########################
    # ToDO: впишите функцию для нахождения ближайшего слова в словаре 
    # (похожим будем считать с расстоянием до max_dist включительно)
    # среди словарных слов с одинаковым расстоянием выбирайте наиболее частотное
    # помните, что словарь упорядочен по частоте
    
    ans = token #если не сможем найти похожее слово, то выведем исходный токен
    min_dist = 1 #если токен не в словаре, то для него минимальное расстояние Д-Л будет равно 1
    dist = max_dist + 1 #значение, куда будем записывать текущее расстояние Д-Л
    len_t = len(token)
    
    for s in freq_dict:
        if (abs(len(s[0]) - len_t) <= max_dist): #разница в длине слова и токена не должна превышать max_dist
            dld = damerau_levenshtein_distance(s[0], token, transposition=True)
            if (dld <= max_dist and dld < dist):
                ans = s[0]
                dist = dld
            if (dist == min_dist):
                return ans
    return ans

In [8]:
%%time

[dummy_spellcorrection(word) for word in wrong]

CPU times: user 906 ms, sys: 0 ns, total: 906 ms
Wall time: 930 ms


['вот',
 'в',
 'инете',
 'отказал',
 'такую',
 'интересную',
 'статейку',
 'предлагаю',
 'вашему',
 'внимание']

### 2. Генеративный способ

Вместо того, чтобы сверять слово, не найденное в словаре, со словарными путем нахождения редакционного или другого расстояния, генерируем для него различные модификации и сравниваем их со словарными.

In [41]:
def norvig_spellcorrection(token): 
    if in_dict(token):
        return token
    
    return max(candidates(token), key=lambda s: s[1])

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(word for word in words if in_dict(word))

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [42]:
%%time

[norvig_spellcorrection(word) for word in wrong]

CPU times: user 33.4 s, sys: 31.2 ms, total: 33.4 s
Wall time: 33.8 s


['вот',
 'в',
 'инете',
 'откопал',
 'такую',
 'интересную',
 'статейку',
 'предлагаю',
 'вашему',
 'внимания']

### Одно из готовых решений: JamSpell 

Учитывает контекст. Как это реализовано, можно почитать у автора: https://habr.com/ru/post/346618/

In [43]:
! apt install swig3.0

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [44]:
! pip install -U jamspell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 215.6 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [52 lines of output]
      /home/peter/p/tf-demo/lib/python3.9/site-packages/setuptools/dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      running bdist_wheel
      running build
      running build_ext
      building '_jamspell' extension
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-639frv8o/jamspell_731ed75ae0434d8b8df54d7b941e98c7/setup.py", line 55, in <module>
          setup(
        File "/home/peter/p/tf-demo/lib/python3.9/site-packages/setuptools/__init_

In [45]:
! wget -q https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz && tar -xvf ru.tar.gz

ru_small.bin


In [46]:
import jamspell

corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('ru_small.bin')

corrector.FixFragment(wrong_text)

ModuleNotFoundError: No module named 'jamspell'

In [49]:
#!pip install pyaspeller
from pyaspeller import YandexSpeller
speller = YandexSpeller()
fixed = speller.spelled(wrong_text)
print(fixed)

вот в инете откопал такую интересную статейку предлагаю вашему вниманию


[Здесь](http://docs.deeppavlov.ai/en/master/features/models/spelling_correction.html#comparison) можно посмотреть на текущее положение дел в коррекции опечаток на русском языке.

## Домашнее задание (6 баллов)

Расстояние Дамерау-Левенштейна, как и многие другие методы нахождения расстояния  между строками, реализовано в различных библиотеках Python. Однако даже реализации  на Cython работают слишком медленно, чтобы их было удобно использовать в прикладных  задачах с большими словарями.

Часть словаря, которая точно не подходит в качестве кандидата для замены неверно написанного слова, можно отфильтровать, прибегнув к векторизации токенов. Если косинусная близость между символьными векторами двух слов небольшая, между ними априори нет смысла вычислять символьное расстояние – если мы ищем кандидата на замену слову "карова", "пешеход" и "ясень" явно не подойдут.

Задача: 
 1. Реализовать метод векторизации токенов, используя информацию о содержащихся в них символьных n-граммах; 
 2. На примере одного предложения на русском языке с несколькими опечатками продемонстрировать время работы функции ``dummy_spellcorrection`` в исходном виде (с поиском по всему словарю) и с фильтрацией по близости векторов слов словаря к вектору слова с опечаткой.

Решения в форме ноутбуков, где последняя ячейка содержит ответ с текстовым комментарием, присылайте на mipttextanalysis20@gmail.com (ссылкой на colab.research или файлом).

Решения без штрафа (8 баллов максимум) принимаются до 12.00 29 сентября. Всё, что присылается не в срок, оценивается из максимума 4 балла. Работу, присланную в срок и оцененную не менее чем на 4 балла, можно доделать и досдать на максимум один раз.

Пожалуйста, указывайте фамилию в названии блокнота. Комментарий к решению принимается на русском или английском языке.

In [9]:
wordlist = []
for s in freq_dict:
    wordlist.append(s[0])

In [10]:
wordlist_l = len(wordlist)
print(wordlist_l)

480092


In [11]:
wrong_sentence = "По мере того как раскрывались перид ней фазисы жызни то есть чуства она зорко наблюдала явления чутко прислушивалась к голасу своево инстинкта и слехка повиряла с нимногими бывшими у ней в запассе наблюдениями и шла астарожно пытая ногой почву на катторую пристояло ступить"
w_sent = wrong_sentence.split()
len_wsent = len(w_sent)
print(w_sent)
print(len_wsent)

['По', 'мере', 'того', 'как', 'раскрывались', 'перид', 'ней', 'фазисы', 'жызни', 'то', 'есть', 'чуства', 'она', 'зорко', 'наблюдала', 'явления', 'чутко', 'прислушивалась', 'к', 'голасу', 'своево', 'инстинкта', 'и', 'слехка', 'повиряла', 'с', 'нимногими', 'бывшими', 'у', 'ней', 'в', 'запассе', 'наблюдениями', 'и', 'шла', 'астарожно', 'пытая', 'ногой', 'почву', 'на', 'катторую', 'пристояло', 'ступить']
43


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer = 'char', ngram_range=(2,2))
dictionary = cv.fit_transform(wordlist)
sentence = cv.transform(w_sent)

In [28]:
print(len(dictionary[0].toarray()[0]))

4507


In [13]:
import numpy as np

X = np.zeros((480092, 4507), np.int8)

for i in range(0, 480092):
    X[i] = dictionary[i].toarray()[0]

In [14]:
print(X.shape)
print(X[0][5])

(480092, 4507)
0


In [15]:
sent_v = np.zeros((43,4507), np.int8)
for i in range(0, 43):
    sent_v[i] = sentence[i].toarray()[0]

In [27]:
print(sent_v[0])

[0 0 0 ... 0 0 0]


In [17]:
from scipy.spatial import distance as d
def in_dict(token):
    for s in freq_dict:
        if s[0] == token:
            return token
            
def dummy_spellcorrection2(token, token_v, max_dist=2, C=0.3):
    if in_dict(token):
        return token
    
    ans = token #если не сможем найти похожее слово, то выведем исходный токен
    min_dist = 1 #если токен не в словаре, то для него минимальное расстояние Д-Л будет равно 1
    dist = max_dist + 1 #значение, куда будем записывать текущее расстояние Д-Л
    len_t = len(token)
    i = 0
    while i < wordlist_l:
        if (abs(len(freq_dict[i][0]) - len_t) <= max_dist):
            if (d.cosine(X[i], token_v) < C):
                dld = damerau_levenshtein_distance(freq_dict[i][0], token, transposition=True)
                if (dld <= max_dist and dld < dist):
                    ans = freq_dict[i][0]
                    dist = dld
                if (dist == min_dist):
                    return ans
        i+=1
    return ans

In [18]:
%%time
i = 0
l = len(w_sent)
ans = []
while i < l:
    ans.append(dummy_spellcorrection2(w_sent[i], sent_v[i], 2, 0.6))
    i+=1
print(ans)

/home/peter/p/tf-demo/lib/python3.9/site-packages/scipy/spatial/distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


['по', 'мере', 'того', 'как', 'раскрывались', 'перед', 'ней', 'оазисы', 'жизни', 'то', 'есть', 'чувства', 'она', 'зорко', 'наблюдала', 'явления', 'чутко', 'прислушивалась', 'к', 'голосу', 'своего', 'инстинкта', 'и', 'слегка', 'потеряла', 'с', 'немногими', 'бывшими', 'у', 'ней', 'в', 'запасе', 'наблюдениями', 'и', 'шла', 'осторожно', 'пытая', 'ногой', 'почву', 'на', 'которую', 'простояло', 'ступить']
CPU times: user 53.6 s, sys: 3.34 s, total: 57 s
Wall time: 59.4 s


In [19]:
%%time

[dummy_spellcorrection(word, 2) for word in w_sent]

CPU times: user 1min 11s, sys: 188 ms, total: 1min 12s
Wall time: 1min 14s


['по',
 'мере',
 'того',
 'как',
 'раскрывались',
 'перед',
 'ней',
 'оазисы',
 'жизни',
 'то',
 'есть',
 'чувства',
 'она',
 'зорко',
 'наблюдала',
 'явления',
 'чутко',
 'прислушивалась',
 'к',
 'голосу',
 'своего',
 'инстинкта',
 'и',
 'слегка',
 'потеряла',
 'с',
 'немногими',
 'бывшими',
 'у',
 'ней',
 'в',
 'запасе',
 'наблюдениями',
 'и',
 'шла',
 'осторожно',
 'пытая',
 'ногой',
 'почву',
 'на',
 'которую',
 'простояло',
 'ступить']

Пример для max_dist = 5:

In [25]:
%%time
i = 0
l = len(w_sent)
ans = []
while i < l:
    ans.append(dummy_spellcorrection2(w_sent[i], sent_v[i], 5, 0.6))
    i+=1
print(ans)

['по', 'мере', 'того', 'как', 'раскрывались', 'перед', 'ней', 'оазисы', 'жизни', 'то', 'есть', 'чувства', 'она', 'зорко', 'наблюдала', 'явления', 'чутко', 'прислушивалась', 'к', 'голосу', 'своего', 'инстинкта', 'и', 'слегка', 'потеряла', 'с', 'немногими', 'бывшими', 'у', 'ней', 'в', 'запасе', 'наблюдениями', 'и', 'шла', 'осторожно', 'пытая', 'ногой', 'почву', 'на', 'которую', 'простояло', 'ступить']
CPU times: user 1min 30s, sys: 9.2 s, total: 1min 39s
Wall time: 1min 49s


In [26]:
%%time

[dummy_spellcorrection(word, 5) for word in w_sent]

CPU times: user 1min 54s, sys: 125 ms, total: 1min 54s
Wall time: 1min 55s


['по',
 'мере',
 'того',
 'как',
 'раскрывались',
 'перед',
 'ней',
 'оазисы',
 'жизни',
 'то',
 'есть',
 'чувства',
 'она',
 'зорко',
 'наблюдала',
 'явления',
 'чутко',
 'прислушивалась',
 'к',
 'голосу',
 'своего',
 'инстинкта',
 'и',
 'слегка',
 'потеряла',
 'с',
 'немногими',
 'бывшими',
 'у',
 'ней',
 'в',
 'запасе',
 'наблюдениями',
 'и',
 'шла',
 'осторожно',
 'пытая',
 'ногой',
 'почву',
 'на',
 'которую',
 'простояло',
 'ступить']

### Комментарий

Оригинальное предложение: По мере того, как раскрывались перед ней фазисы жизни, то есть чувства, она зорко наблюдала явления, чутко прислушивалась к голосу своего инстинкта и слегка поверяла с немногими, бывшими у ней в запасе наблюдениями, и шла осторожно, пытая ногой почву, на которую предстояло ступить. (И. А Гончаров "Обломов")  

Векторизацию я выполнил при помощи CountVectorizer для символьных биграмм, поскольку размер векторов для триграмм и т.д. становится слишком большим. На данном предложении фильтрация по близости векторов слов словаря к вектору слова с опечаткой показала себя быстрее, чем поиск по всему словарю, но ускорение не очень большое (~10-15 сек при различных значениях max_dist). Предполагаю, что на бОльших предложениях и с более длинными словами ускорение может быть значительно выше.